In [3]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import pandas as pd
import numpy as np
import random
from torch_geometric.utils import from_scipy_sparse_matrix
import scipy.sparse as sp
from scipy.spatial import distance_matrix
from torch_geometric.data import Data
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn.models import InnerProductDecoder, VGAE, GAE
from torch_geometric.nn.conv import GCNConv, GINConv, SAGEConv, SGConv, GATConv
from torch_geometric.nn import APPNP as APPNPConv
import warnings
warnings.filterwarnings('ignore')

In [6]:
class MLP(nn.Module):
    def __init__(self, input_dim:int, hidden_dim:int, output_dim:int, dropout:0.):
        super(MLP, self).__init__()
        self.lin1 = nn.Linear(input_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)
        # self.relu = nn.ReLU()
        self.lin2 = nn.Linear(hidden_dim, output_dim)
        self.reset_parameters()
        
    def reset_parameters(self):
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()
        
    def forward(self, x, edge_index, edge_weight=None):
        x = self.lin1(x)
        # x = self.relu(x)
        x = self.dropout(x)
        x = self.lin2(x)
        # x = self.relu(x)
        return x

In [7]:
class GCN(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim, dropout=0.5):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_dim, hid_dim, dropout)
        # self.conv2 = GCNConv(hid_dim, hid_dim, dropout) # delete
        self.lin = nn.Linear(hid_dim, out_dim)
        # self.dropout = nn.Dropout(dropout) #delete
        
        for m in self.modules():
            self.weights_init(m)
        
    def weights_init(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform_(m.weight.data)
            if m.bias is not None:
                m.bias.data.fill_(0.0)
        
    def forward(self, x, edge_index, edge_weight=None):
        x = self.conv1(x, edge_index, edge_weight)
        # x = self.dropout(x)                                 # delete
        # x = self.conv2(x, edge_index, edge_weight)            # delete
        x = self.lin(x)
        return x

In [ ]:
# SAGEConv
class GraphSAGE(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim, dropout=0.5):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_dim, hid_dim, normalize=True)
        self.conv2 = SAGEConv(hid_dim, hid_dim, normalize=True)
        self.lin = nn.Linear(hid_dim, out_dim)
        self.transition = nn.Sequential(
            nn.ReLU(),
            nn.BatchNorm1d(hid_dim),
            nn.Dropout(dropout)
        )
        
        for m in self.modules():
            self.weights_init(m)

    def weights_init(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform_(m.weight.data)
            if m.bias is not None:
                m.bias.data.fill_(0.0)
    
    def forward(self, x, edge_index, edge_weight=None):
        x = self.conv1(x, edge_index, edge_weight)
        x = self.transition(x)
        x = self.conv2(x, edge_index, edge_weight)
        x = self.lin(x)
        return x

In [ ]:
# APPNP
class APPNP(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim, dropout=0.5):
        super(APPNP, self).__init__()
        self.lin1 = nn.Linear(in_dim, hid_dim)
        self.lin2 = nn.Linear(hid_dim, out_dim)
        self.relu = nn.ReLU()
        self.prop1 = APPNPConv(K=2, alpha=0.1)
        #self.prop2 = APPNPConv(K=2, alpha=0.1)
        self.dropout = nn.Dropout(dropout)
        self.reset_parameters()
    
    def reset_parameters(self):
        self.prop1.reset_parameters()
        # self.prop2.reset_parameters()
        
    def forward(self, x, edge_index, edge_weight=None):
        x = self.lin1(x)
        x = self.dropout(x)
        x = self.prop1(x, edge_index, edge_weight)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.lin2(x)
        # x = self.dropout(x)
        # x = self.prop(x, edge_index, edge_weight)
        return x

In [ ]:
class GIN(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim, dropout=0.5):
        super(GIN, self).__init__()
        self.conv1 = GINConv(nn.Sequential(nn.Linear(in_dim, hid_dim),
                             nn.ReLU(), nn.BatchNorm1d(hid_dim),
                             nn.Linear(hid_dim, hid_dim)))
        #self.conv2 = GINConv(nn.Sequential(nn.Linear(hid_dim, hid_dim),
         #                         nn.ReLU()), nn.BatchNorm1d(hid_dim))
        self.lin = nn.Linear(hid_dim, out_dim)
    
        for m in self.modules():
            self.weights_init(m)

    def weights_init(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform_(m.weight.data)
            if m.bias is not None:
                m.bias.data.fill_(0.0)
        
    def forward(self, x, edge_index, edge_weight=None):
        x = self.conv1(x, edge_index, edge_weight)
        #x = self.conv2(x, edge_index, edge_weight)
        x = self.lin(x)
        return x

In [5]:
# SGConv
class SGC(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim, dropout=0.5):
        super(SGC, self).__init__()
        self.conv1 = SGConv(in_dim, hid_dim, k=2)
        self.conv2 = SGConv(hid_dim, out_dim, k=1)
        # self.lin = nn.Linear(hid_dim, out_dim)
        self.reset_parameters()
    
    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        
    
    def forward(self, x, edge_index, edge_weight=None):
        x = self.conv1(x, edge_index, edge_weight)
        x = self.conv2(x, edge_index, edge_weight)
        return x

In [ ]:
class GAT(torch.nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim, dropout=0.5):
        super().__init__()
        self.conv1 = GATConv(in_dim, hid_dim, heads=8)
        # On the Pubmed dataset, use `heads` output heads in `conv2`.
        # self.conv2 = GATConv(hid_dim * 8, hid_dim, heads=1, concat=False)
        self.lin = nn.Linear(hid_dim, out_dim)
    
        for m in self.modules():
            self.weights_init(m)

    def weights_init(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform_(m.weight.data)
            if m.bias is not None:
                m.bias.data.fill_(0.0)
        
    def forward(self, x, edge_index, edge_weight=None):
        # torch.use_deterministic_algorithms(False)
        x = self.conv1(x, edge_index, edge_weight)
        # x = self.conv2(x, edge_index, edge_weight)
        x = self.lin(x)
        return x